In [ ]:
sentences = []
for image,result in tqdm(results.items()):
    dim = tuple(reversed(result.pages[0].dimensions))
    predictions = []
    block_id = 0
    for block in result.pages[0].blocks:
        sentence = []
        line_id = 0
        for line in block.lines: 
            for word in line.words:
                sentence.append(word.value)
        sentences.append(' '.join(sentence))

sentences[136]



import xml.etree.ElementTree as ET

def doctr_to_hocr(doc, image_path, page_num=1):
    # Create an XML element for the HTML page
    page = ET.Element('html')
    page.set('xmlns', 'http://www.w3.org/1999/xhtml')

    # Create an XML element for the page body
    body = ET.SubElement(page, 'body')

    # Add the OCR results to the page body
    for block in doc.blocks:
        block_elem = ET.SubElement(body, 'div')
        block_elem.set('class', 'ocr_carea')
        for line in block.lines:
            line_elem = ET.SubElement(block_elem, 'span')
            line_elem.set('class', 'ocr_line')
            for word in line.words:
                word_elem = ET.SubElement(line_elem, 'span')
                word_elem.set('class', 'ocrx_word')
                word_elem.text = word.value

    # Add the image to the page
    image_elem = ET.SubElement(body, 'img')
    image_elem.set('src', image_path)
    image_elem.set('alt', '')
    image_elem.set('class', 'ocr_page')
    image_elem.set('title', 'image ' + image_path + '; page ' + str(page_num))
#     image_elem.set('title', 'image {}; page {}', format(image_path, str(page_num)))

    # Serialize the XML element to a string and return it
    hocr = ET.tostring(page, encoding='unicode')
    return hocr


def doctr_to_hocr_with_bbox(doc, image_path, iou_thresh=0.5):
    # Initialize the HOCR string with the HTML header and the OCR page element
    hocr = '''<!DOCTYPE html>
<html>
<head>
<title></title>
<meta http-equiv="Content-Type" content="text/html;charset=utf-8" />
<meta name='ocr-system' content='doctr' />
</head>
<body>
<div class='ocr_page' id='page_1' title='image "{0}"'>
'''.format(image_path)

    # Add the OCR line and word elements to the HOCR string
    for page_num, page in enumerate(doc.pages):
        # Initialize the text and bbox lists for the page
        text_list = []
        bbox_list = []
        
        for block in page.blocks:
            # Initialize the text and bbox lists for the block
            block_text_list = []
            block_bbox_list = []
            
            for line in block.lines:
                # Initialize the text and bbox lists for the line
                line_text_list = []
                line_bbox_list = []
                
                for word in line.words:
                    # Add the text and bbox of the word to the line lists
                    line_text_list.append(word.value)
                    line_bbox_list.append(word.geometry.bbox)
                    
                # Combine the text and bbox lists for the line into a single string
                line_text = ' '.join(line_text_list)
                line_bbox = ' ; '.join(['{} {} {} {}'.format(*bbox) for bbox in line_bbox_list])

                # Add the OCR line element to the HOCR string
                line_elem = '<span class="ocr_line" title="bbox {}" >{}</span>'.format(line_bbox, line_text)
                block_text_list.append(line_text)
                block_bbox_list.append(line_bbox)
                hocr += line_elem + '\n'
            
            # Combine the text and bbox lists for the block into a single string
            block_text = ' '.join(block_text_list)
            block_bbox = ' ; '.join(['{} {} {} {}'.format(*bbox) for bbox in block_bbox_list])

            # Add the OCR block element to the HOCR string
            block_elem = '<div class="ocr_carea" title="bbox {}">{}</div>'.format(block_bbox, block_text)
            hocr += block_elem + '\n'
            
            # Add the text and bbox of the block to the page lists
            text_list.append(block_text)
            bbox_list.append(block_bbox)

        # Combine the text and bbox lists for the page into a single string
        page_text = ' '.join(text_list)
        page_bbox = ' ; '.join(['{} {} {} {}'.format(*bbox) for bbox in bbox_list])
        
        # Add the OCR page element to the HOCR string
        image_elem = '<img class="ocr_page" src="{}" title="image {}; page {}" alt="" />'.format(image_path, image_path, page_num+1)
        page_elem = '<div class="ocr_page" id="page_{}" title="image {}; bbox {}" >{}{}</div>'.format(
            page_num+1, image_path, page_bbox, image_elem, page_text)
        hocr += page_elem + '\n'

    # Add the HTML footer to the HOCR string
    hocr += '</div>\n</body>\n</html>'

    return hocr



hocr = doctr_to_hocr_with_bbox(result, image_file, 0.5)

def save_hocr(hocr, output_path):
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(hocr)

from bs4 import BeautifulSoup

def hocr_to_html(hocr):
    # Parse the HOCR string as HTML using BeautifulSoup
    soup = BeautifulSoup(hocr, 'html.parser')

    # Remove HOCR-specific metadata from the HTML
    for tag in soup(['ocrx_word', 'ocr_line', 'ocr_carea', 'ocr_par', 'ocr_page']):
        tag.unwrap()

    # Return the cleaned HTML
    return str(soup)

html = hocr_to_html(hocr)

save_hocr(html, 'sample.html')